# Momentum, Volatility, and Volume Factors in U.S. Stock Returns

**ISYE 4031 Final Project**  
*Industrial & Systems Engineering, Georgia Tech*

## 📊 Project Overview

This notebook analyzes the relationship between **momentum**, **volatility**, and **volume** factors in U.S. stock returns using S&P 500 data.

### Research Questions:
1. Do momentum indicators significantly predict future stock returns?
2. How does volatility clustering affect return predictability? 
3. Is trading volume a reliable indicator of price direction?

---

In [2]:
import yfinance as yf
import pandas as pd
from pandas_datareader import data as pdr
import datetime as dt
import numpy as np
from bs4 import BeautifulSoup
import requests, re

## 📈 Data Collection

### Step 1: S&P 500 Stock List
We start by scraping the current S&P 500 stock list from a reliable financial data source.

**Data Source**: [Stock Analysis - S&P 500](https://stockanalysis.com/list/sp-500-stocks/)

**Key Information Collected**:
- Stock symbols (tickers)
- Market capitalization
- Current stock prices
- Recent price changes

In [12]:
url = 'https://stockanalysis.com/list/sp-500-stocks/'
resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'html.parser')

# Find the table and extract headers
table = soup.find('table', class_='symbol-table svelte-1ro3niy')
headers = [th.get_text(strip=True) for th in table.find('tr').find_all('th')]

# Extract all row data
stocks_data = []
for row in table.find_all('tr')[1:]:  # Skip header row
    row_data = [cell.get_text(strip=True) for cell in row.find_all('td')]
    stocks_data.append(row_data)

# Create DataFrame and set No. column as index
sp500_df = pd.DataFrame(stocks_data, columns=headers)
sp500_df = sp500_df.set_index('No.')

print("\nFirst 10 rows:")
print(sp500_df.head(10)[['Symbol', 'Market Cap', 'Stock Price', '% Change']])


First 10 rows:
    Symbol Market Cap Stock Price % Change
No.                                       
1     NVDA      4.57T      188.15    0.04%
2     AAPL      3.97T      268.47   -0.48%
3     MSFT      3.69T      496.82   -0.06%
4     GOOG      3.37T      279.70   -1.98%
5    GOOGL      3.36T      278.83   -2.08%
6     AMZN      2.61T      244.41    0.56%
7     AVGO      1.65T      349.43   -1.73%
8     META      1.57T      621.71    0.45%
9     TSLA      1.43T      429.52   -3.68%
10   BRK.B      1.08T      499.06    1.20%


In [13]:
# Download stock data for S&P 500 companies
stocks = sp500_df['Symbol'].tolist()  # Convert to list
endDate = dt.datetime(2024, 12, 31)
startDate = dt.datetime(2021, 1, 1)

print(f"Downloading data for {len(stocks)} stocks from {startDate.date()} to {endDate.date()}...")
print("This may take a few minutes...")

# Download with error handling
try:
    df = yf.download(stocks, start=startDate, end=endDate, group_by='ticker', auto_adjust=True, prepost=True, threads=True)
    print(f"✅ Successfully downloaded data!")
    print(f"DataFrame shape: {df.shape}")
    print(f"Date range: {df.index[0]} to {df.index[-1]}")
    display(df.head())
except Exception as e:
    print(f"❌ Error downloading data: {e}")
    # Try with a smaller subset if full download fails
    print("Trying with first 10 stocks...")
    df = yf.download(stocks[:10], start=startDate, end=endDate, group_by='ticker', auto_adjust=True)
    display(df.head())

This may take a few minutes...


[*********************100%***********************]  502 of 502 completed

2 Failed downloads:
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2021-01-01 00:00:00 -> 2024-12-31 00:00:00)')
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')


✅ Successfully downloaded data!
DataFrame shape: (1004, 2512)
Date range: 2021-01-04 00:00:00 to 2024-12-30 00:00:00


Ticker           MNST                                                 IBKR  \
Price            Open       High        Low      Close   Volume       Open   
Date                                                                         
2021-01-04  46.255001  46.275002  44.400002  45.299999  6013200  14.966611   
2021-01-05  44.945000  45.830002  44.724998  45.709999  5060200  14.922797   
2021-01-06  45.529999  45.700001  45.044998  45.349998  7577800  15.429068   
2021-01-07  45.285000  46.799999  45.134998  46.750000  6040800  16.441612   
2021-01-08  46.889999  47.529999  46.474998  47.494999  6277600  16.874858   

Ticker                                                ...           A  \
Price            High        Low      Close   Volume  ...        Open   
Date                                                  ...               
2021-01-04  15.090744  14.601513  14.903327  2736800  ...  115.155142   
2021-01-05  15.421765  14.871683  15.373085  4374400  ...  114.709786   
2021-01-06  16.680139  15.426634  16.285833  7665600  ...  114.913100   
2021-01-07  17.188848  16.327214  16.838352  6076400  ...  120.373618   
2021-01-08  17.493092  16.794536  17.008726  4040800  ...  122.464878   

Ticker                                                         NUE             \
Price             High         Low       Close   Volume       Open       High   
Date                                                                            
2021-01-04  116.268541  113.383373  114.864685  2030700  50.290837  50.727100   
2021-01-05  116.491231  114.312830  115.803825  2344000  48.749988  50.615704   
2021-01-06  119.937941  114.448371  118.979446  2428500  52.314348  53.418926   
2021-01-07  123.423381  120.160617  122.145386  1775900  53.502472  53.604576   
2021-01-08  123.578284  121.845244  123.016739  1577200  52.175109  52.537113   

Ticker                                     
Price             Low      Close   Volume  
Date                                       
2021-01-04  48.490096  48.740711  2308900  
2021-01-05  48.703578  50.188725  2223500  
2021-01-06  51.683160  52.926971  3291300  
2021-01-07  51.673882  52.156555  2229400  
2021-01-08  50.829193  51.998749  2720800  

[5 rows x 2512 columns]

### Step 2: Historical Stock Data Download

Now we download **4 years** of daily stock data (2021-2024) for our analysis.

**Data Parameters**:
- 📅 **Time Period**: January 1, 2021 → December 31, 2024
- 📊 **Data Fields**: Open, High, Low, Close, Volume, Adjusted Close  
- 🏢 **Companies**: Top S&P 500 stocks
- 🔄 **Update Frequency**: Daily

> **Note**: We're using a subset of stocks for faster processing and testing.